In [0]:
%pip install -U -qqqq databricks-langchain[memory] uv databricks-agents mlflow-skinny[databricks]
dbutils.library.restartPython()

In [0]:
%load_ext autoreload
%autoreload 2
import warnings

warnings.simplefilter(action="ignore")

# Mosaic AI Agent Framework: Author and deploy a Stateful Agent using Databricks Lakebase and LangGraph
This notebook demonstrates how to build a stateful agent using the Mosaic AI Agent Framework and LangGraph, with Lakebase as the agent’s durable memory and checkpoint store. In this notebook, you will:
1. Author a Stateful Agent graph with LakeBase (the new Postgres database in Databricks) and Langgraph to manage state using thread ids in a Databricks Agent 
2. Wrap the LangGraph agent with MLflow ChatAgent to ensure compatibility with Databricks features
3. Test the agent's behavior locally
4. Register model to Unity Catalog, log and deploy the agent for use in apps and Playground

We use [PostgresSaver in Langgraph](https://api.python.langchain.com/en/latest/checkpoint/langchain_postgres.checkpoint.PostgresSaver.html) to open a connection with our Lakebase, pass it into the checkpoint and pass that into the LangGraph Agent

## Why use Lakebase?
Stateful agents need a place to persist, resume, and inspect their work. Lakebase provides a managed, UC-governed store for agent state:
- Durable, resumable state. Automatically capture threads, intermediate checkpoints, tool outputs, and node state after each graph step—so you can resume, branch, or replay any point in time.
- Queryable & observable. Because state lands in the Lakehouse, you can use SQL (or notebooks) to audit conversations and build upon other Databricks functionality like dashboards
- Governed by Unity Catalog. Apply data permissions, lineage, and auditing to AI state, just like any other table.

## What are Stateful Agents?
Unlike stateless LLM calls, a stateful agent keeps and reuses context across steps and sessions. Each new conversation is tracked with a thread ID, which represents the logical task or dialogue stream. This way, you can pick up an existing thread and continue the conversation with your Agent.

## Prerequisites
- Create a Lakebase instance, see Databricks documentation ([AWS](https://docs.databricks.com/aws/en/oltp/create/) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/oltp/create/)). 
- You can create a Lakebase instance by going to SQL Warehouses -> Lakebase Postgres -> Create database instance. You will need to retrieve values from the "Connection details" section of your Lakebase to fill out this notebook.

In [0]:
dbutils.widgets.text(name="catalog", defaultValue="", label="catalog")
dbutils.widgets.text(name="schema", defaultValue="", label="schema")
dbutils.widgets.text(name="model", defaultValue="", label="model")
dbutils.widgets.text(
    name="DATABRICKS_CLIENT_ID", defaultValue="", label="DATABRICKS_CLIENT_ID"
)
dbutils.widgets.text(
    name="DATABRICKS_CLIENT_SECRET", defaultValue="", label="DATABRICKS_CLIENT_SECRET"
)
dbutils.widgets.text(name="secret_scope", defaultValue="", label="secret_scope")

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
model = dbutils.widgets.get("model")
# LLM_ENDPOINT = dbutils.widgets.get("foundation_model")
assert (
    len(catalog) > 0 and len(schema) > 0 and len(model) > 0
), "Please provide a valid catalog, schema, and model name"
three_tiered_model_name = f"{catalog}.{schema}.{model}"
print(f"{three_tiered_model_name=}")

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

DATABRICKS_HOST = w.config.host

secret_scope_name = dbutils.widgets.get("secret_scope")

# if needed create a secret scope
if secret_scope_name != "dbdemos":
    w.secrets.create_scope(scope=secret_scope_name)
else:
    print(f"Using existing secret scope: {secret_scope_name}")

In [0]:
if dbutils.widgets.get("DATABRICKS_CLIENT_ID") == "":
    print("no DATABRICKS_CLIENT_ID is provided")
else:
    w.secrets.put_secret(
        scope=secret_scope_name,
        key="DATABRICKS_CLIENT_ID",
        string_value=dbutils.widgets.get("DATABRICKS_CLIENT_ID"),
    )
if dbutils.widgets.get("DATABRICKS_CLIENT_SECRET") == "":
    print("no DATABRICKS_CLIENT_ID is provided")
else:
    w.secrets.put_secret(
        scope=secret_scope_name,
        key="DATABRICKS_CLIENT_SECRET",
        string_value=dbutils.widgets.get("DATABRICKS_CLIENT_SECRET"),
    )
w.secrets.put_secret(
    scope=secret_scope_name, key="DATABRICKS_HOST", string_value=DATABRICKS_HOST
)

In [0]:
# import os

# os.environ["DATABRICKS_CLIENT_ID"] = dbutils.secrets.get(
#     scope=secret_scope_name, key="DATABRICKS_CLIENT_ID"
# )
# os.environ["DATABRICKS_CLIENT_SECRET"] = dbutils.secrets.get(
#     scope=secret_scope_name, key="DATABRICKS_CLIENT_SECRET"
# )

# os.unsetenv("DATABRICKS_CLIENT_ID")
# os.unsetenv("DATABRICKS_CLIENT_SECRET")

## Lakebase Config
- Enable Postgres native role login
- Might need to wait a few min for pg roles to apply
- Create new catalog with PostgreSQL Database: `databricks_postgres` schema off lakebase instance for querying purposes

In [0]:
# First-time checkpoint table setup
from databricks.sdk import WorkspaceClient
from databricks_langchain import CheckpointSaver

# --- TODO: Fill in Lakebase instance name ---
INSTANCE_NAME = "<INSERT LAKEBASE NAME>"

# Create tables if missing
with CheckpointSaver(instance_name=INSTANCE_NAME) as saver:
    saver.setup()  # sets up checkpoint tables
    print("✅ Checkpoint tables are ready.")

In [0]:
%%writefile agent.py
import logging
import os
import uuid
from typing import Annotated, Any, Generator, Optional, Sequence, TypedDict

import mlflow
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    CheckpointSaver,
)
from databricks.sdk import WorkspaceClient
from langchain_core.messages import AIMessage, AIMessageChunk, AnyMessage
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    output_to_responses_items_stream,
)

logger = logging.getLogger(__name__)
logging.basicConfig(level=os.getenv("LOG_LEVEL", "INFO"))

############################################
# Define your LLM endpoint and system prompt
############################################
# TODO: Replace with your model serving endpoint
LLM_ENDPOINT_NAME = "databricks-gpt-5-2"

# TODO: Update with your system prompt
SYSTEM_PROMPT = """
    You are an cybersecurity assistant.
    You are given a task and you must complete it.
    Use the following routine to support the customer.
    # Routine:
    1. Provide the get_cyber_threat_info tool the type of threat being asked about.
    2. Use the source ip address provided in step 1 as input for the get_user_info tool to retrieve user specific info.
    Use the following tools to complete the task:
    {tools}"""

############################################
# Lakebase configuration
############################################
# TODO: Fill in Lakebase instance name
LAKEBASE_INSTANCE_NAME = "<INSERT LAKEBASE NAME>"

###############################################################################
## Define tools for your agent,enabling it to retrieve data or take actions
## beyond text generation
## To create and see usage examples of more tools, see
## https://docs.databricks.com/en/generative-ai/agent-framework/agent-tool.html
###############################################################################
tools = []

# Example UC tools; add your own as needed
UC_TOOL_NAMES: list[str] = [
    "catalog.schema.get_cyber_threat_info",
    "cataog.schema.get_user_info",
]
if UC_TOOL_NAMES:
    uc_toolkit = UCFunctionToolkit(function_names=UC_TOOL_NAMES)
    tools.extend(uc_toolkit.tools)

# Use Databricks vector search indexes as tools
# See https://docs.databricks.com/en/generative-ai/agent-framework/unstructured-retrieval-tools.html#locally-develop-vector-search-retriever-tools-with-ai-bridge
# List to store vector search tool instances for unstructured retrieval.
VECTOR_SEARCH_TOOLS = []

# To add vector search retriever tools,
# use VectorSearchRetrieverTool and create_tool_info,
# then append the result to TOOL_INFOS.
# Example:
# VECTOR_SEARCH_TOOLS.append(
#     VectorSearchRetrieverTool(
#         index_name="",
#         # filters="..."
#     )
# )

tools.extend(VECTOR_SEARCH_TOOLS)

#####################
## Define agent logic
#####################


class AgentState(TypedDict):
    messages: Annotated[Sequence[AnyMessage], add_messages]
    custom_inputs: Optional[dict[str, Any]]
    custom_outputs: Optional[dict[str, Any]]


class LangGraphResponsesAgent(ResponsesAgent):
    """Stateful agent using ResponsesAgent with pooled Lakebase checkpointing."""

    def __init__(self, lakebase_config: dict[str, Any]):
        self.workspace_client = WorkspaceClient()

        self.model = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)
        self.system_prompt = SYSTEM_PROMPT
        self.model_with_tools = self.model.bind_tools(tools) if tools else self.model

    def _create_graph(self, checkpointer: Any):
        def should_continue(state: AgentState):
            messages = state["messages"]
            last_message = messages[-1]
            if isinstance(last_message, AIMessage) and last_message.tool_calls:
                return "continue"
            return "end"

        preprocessor = (
            RunnableLambda(
                lambda state: [{"role": "system", "content": self.system_prompt}]
                + state["messages"]
            )
            if self.system_prompt
            else RunnableLambda(lambda state: state["messages"])
        )
        model_runnable = preprocessor | self.model_with_tools

        def call_model(state: AgentState, config: RunnableConfig):
            response = model_runnable.invoke(state, config)
            return {"messages": [response]}

        workflow = StateGraph(AgentState)
        workflow.add_node("agent", RunnableLambda(call_model))

        if tools:
            workflow.add_node("tools", ToolNode(tools))
            workflow.add_conditional_edges(
                "agent", should_continue, {"continue": "tools", "end": END}
            )
            workflow.add_edge("tools", "agent")
        else:
            workflow.add_edge("agent", END)

        workflow.set_entry_point("agent")
        return workflow.compile(checkpointer=checkpointer)

    def _get_or_create_thread_id(self, request: ResponsesAgentRequest) -> str:
        """Get thread_id from request or create a new one.

        Priority:
        1. Use thread_id from custom_inputs if present
        2. Use conversation_id from chat context if available
        3. Generate a new UUID

        Returns:
            thread_id: The thread identifier to use for this conversation
        """
        ci = dict(request.custom_inputs or {})

        if "thread_id" in ci:
            return ci["thread_id"]

        # using conversation id from chat context as thread id
        # https://mlflow.org/docs/latest/api_reference/python_api/mlflow.types.html#mlflow.types.agent.ChatContext
        if request.context and getattr(request.context, "conversation_id", None):
            return request.context.conversation_id

        # Generate new thread_id
        return str(uuid.uuid4())

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(
            output=outputs, custom_outputs=request.custom_inputs
        )

    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        thread_id = self._get_or_create_thread_id(request)
        ci = dict(request.custom_inputs or {})
        ci["thread_id"] = thread_id
        request.custom_inputs = ci

        # Convert incoming Responses messages to ChatCompletions format
        # LangChain will automatically convert from ChatCompletions to LangChain format
        cc_msgs = self.prep_msgs_for_cc_llm([i.model_dump() for i in request.input])
        langchain_msgs = cc_msgs
        checkpoint_config = {"configurable": {"thread_id": thread_id}}

        with CheckpointSaver(instance_name=LAKEBASE_INSTANCE_NAME) as checkpointer:
            graph = self._create_graph(checkpointer)

            for event in graph.stream(
                {"messages": langchain_msgs},
                checkpoint_config,
                stream_mode=["updates", "messages"],
            ):
                if event[0] == "updates":
                    for node_data in event[1].values():
                        if len(node_data.get("messages", [])) > 0:
                            yield from output_to_responses_items_stream(
                                node_data["messages"]
                            )
                elif event[0] == "messages":
                    try:
                        chunk = event[1][0]
                        if isinstance(chunk, AIMessageChunk) and chunk.content:
                            yield ResponsesAgentStreamEvent(
                                **self.create_text_delta(
                                    delta=chunk.content, item_id=chunk.id
                                ),
                            )
                    except Exception as exc:
                        logger.error("Error streaming chunk: %s", exc)


# ----- Export model -----
mlflow.langchain.autolog()
AGENT = LangGraphResponsesAgent(LAKEBASE_INSTANCE_NAME)
mlflow.models.set_model(AGENT)

In [0]:
from agent import AGENT

result = AGENT.predict(
    {
        "input": [
            {
                "role": "user",
                "content": "Who committed the latest malware threat?",
            }
        ],
    }
)
print(result.model_dump(exclude_none=True))
thread_id = result.custom_outputs["thread_id"]

In [0]:
# Message 2, include thread ID and notice how agent remembers context from previous predict message
response2 = AGENT.predict(
    {
        "input": [{"role": "user", "content": "Who was just mentioned?"}],
        "custom_inputs": {"thread_id": thread_id},
    }
)
print("Response 2:", response2.model_dump(exclude_none=True))

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import (
    DatabricksFunction,
    DatabricksServingEndpoint,
    DatabricksLakebase,
    DatabricksVectorSearchIndex,
)  # we are adding DatabricksLakebase resource type
from mlflow.models.auth_policy import AuthPolicy, SystemAuthPolicy, UserAuthPolicy
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool
from agent import LLM_ENDPOINT_NAME, LAKEBASE_INSTANCE_NAME, tools
from pkg_resources import get_distribution

# TODO: Manually include additional underlying resources if needed and update values for endpoint/lakebase
resources = [
    DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME),
    DatabricksLakebase(database_instance_name=LAKEBASE_INSTANCE_NAME),
]
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

# System policy: resources accessed with system credentials
system_policy = SystemAuthPolicy(resources=resources)

# User policy: API scopes for OBO access
api_scopes = [
    "sql.statement-execution",
    "mcp.genie",
    "mcp.external",
    "catalog.connections",
    "mcp.vectorsearch",
    "vectorsearch.vector-search-indexes",
    "iam.current-user:read",
    "sql.warehouses",
    "dashboards.genie",
    "serving.serving-endpoints",
    "iam.access-control:read",
    "apps.apps",
    "mcp.functions",
    "vectorsearch.vector-search-endpoints",
]
user_policy = UserAuthPolicy(api_scopes=api_scopes)

input_example = {
    "input": [{"role": "user", "content": "What is an LLM agent?"}],
    "custom_inputs": {"thread_id": "example-thread-123"},
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        pip_requirements=[
            f"databricks-langchain[memory]=={get_distribution('databricks-langchain[memory]').version}",
        ],
        resources=resources,
    )

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data=input_example,
    env_manager="uv",
)

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
UC_MODEL_NAME = f"{catalog}.{schema}.{model}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri,
    name=UC_MODEL_NAME,
)

In [0]:
from databricks import agents

agents.deploy(
    UC_MODEL_NAME,
    uc_registered_model_info.version,
    environment_vars={
        "DATABRICKS_HOST": "{{secrets/secrets_scope/DATABRICKS_HOST}}",
        "DATABRICKS_CLIENT_ID": "{{secrets/secrets_scope/DATABRICKS_CLIENT_ID}}",
        "DATABRICKS_CLIENT_SECRET": "{{secrets/secrets_scope/DATABRICKS_CLIENT_SECRET}}",
    },
    tags={"endpointSource": "playground"},
)

In [0]:
from databricks import agents

# Note that  can specify individual users or groups.
agents.set_permissions(
    model_name=UC_MODEL_NAME,
    users=["users"],
    permission_level=agents.PermissionLevel.CAN_QUERY,
)

## Next steps
After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See docs for details

In [0]:
import time
from databricks.sdk.service.serving import EndpointStateReady, EndpointStateConfigUpdate
from databricks.sdk import WorkspaceClient

endpoint_name: str = f"agents_{catalog}-{schema}-{model}"
print("\nWaiting for endpoint to deploy.  This can take 10 - 20 minutes.", end="")
w = WorkspaceClient()
while (
    w.serving_endpoints.get(endpoint_name).state.ready == EndpointStateReady.NOT_READY
    or w.serving_endpoints.get(endpoint_name).state.config_update
    == EndpointStateConfigUpdate.IN_PROGRESS
):
    print(".", end="")
    time.sleep(30)

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
ep = w.serving_endpoints.get(endpoint_name)
print(ep.task)

In [0]:
from mlflow.deployments import get_deploy_client

w = WorkspaceClient()
endpoint_name: str = f"agents_{catalog}-{schema}-{model}"
res = get_deploy_client("databricks").predict(
    endpoint=endpoint_name,
    inputs={
        "input": [
            {
                "role": "user",
                "content": "Who did I just mention?",
            }
        ],
        "max_tokens": 400,
        "custom_inputs": {"thread_id": thread_id},
        "temperature": 0.1,
    },
)

In [0]:
if "output" in res:
    print(res["output"][0]["content"][-1]["text"])